# BharatBuild Qwen Fine-tuning (Kaggle)

**GPU:** T4 x2 (Free)
**Time:** ~3-4 hours
**Cost:** FREE

In [ ]:
# Cell 1: Check GPU
import torch
print(f"GPUs available: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")

In [ ]:
# Cell 2: Install dependencies
!pip install -q transformers>=4.37.0 datasets>=2.16.0 accelerate>=0.25.0
!pip install -q peft>=0.7.0 bitsandbytes>=0.41.0 trl>=0.7.0
!pip install -q scipy sentencepiece

In [ ]:
# Cell 3: Upload training data
# Click the '+' icon on the right panel -> Upload -> Select train.jsonl
# Or drag and drop your file to the Input section

# After upload, your file will be at:
# /kaggle/input/your-dataset-name/train.jsonl

# For now, let's upload directly:
from IPython.display import display, HTML
display(HTML('<h3>Upload train.jsonl using the panel on the right (+ Add Data)</h3>'))

import os
# Check if file exists
if os.path.exists('/kaggle/input'):
    print("Input folder contents:")
    !ls -la /kaggle/input/
else:
    print("No input data yet. Upload your train.jsonl file.")

In [ ]:
# Cell 4: Set the data file path
# IMPORTANT: Update this path after uploading your file!

# Option A: If you uploaded as a dataset:
# DATA_FILE = "/kaggle/input/your-dataset-name/train.jsonl"

# Option B: Upload directly to working directory:
# Run this to upload:
from google.colab import files
try:
    uploaded = files.upload()
    DATA_FILE = "/kaggle/working/train.jsonl"
    !mv train.jsonl /kaggle/working/ 2>/dev/null || true
except:
    # If not in Colab-compatible mode, set path manually
    DATA_FILE = "/kaggle/input/training-data/train.jsonl"  # Update this!
    print(f"Set DATA_FILE to: {DATA_FILE}")
    print("Update the path above if different!")

In [ ]:
# Cell 5: Configuration
import os
import torch
from datetime import datetime
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer, SFTConfig

# Model
MODEL_NAME = "Qwen/Qwen2.5-Coder-7B-Instruct"
OUTPUT_DIR = "/kaggle/working/qwen-bharatbuild"

# Training params - optimized for Kaggle T4
BATCH_SIZE = 2
GRADIENT_ACCUMULATION = 8
LEARNING_RATE = 2e-4
NUM_EPOCHS = 3
MAX_SEQ_LENGTH = 1024  # Reduced for speed

# LoRA
LORA_R = 32  # Reduced for faster training
LORA_ALPHA = 64
LORA_DROPOUT = 0.05

print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

In [ ]:
# Cell 6: Load tokenizer and model
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True, padding_side="right")
tokenizer.pad_token = tokenizer.eos_token

print("Configuring 4-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading model: {MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)
print("Model loaded!")

In [ ]:
# Cell 7: Apply LoRA
print("Applying LoRA...")
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Cell 8: Load and format dataset
def format_training_sample(example):
    messages = example['messages']
    system_msg = ""
    user_msg = ""
    assistant_msg = ""
    for msg in messages:
        if msg['role'] == 'system':
            system_msg = msg['content']
        elif msg['role'] == 'user':
            user_msg = msg['content']
        elif msg['role'] == 'assistant':
            assistant_msg = msg['content']
    return {"text": "<|im_start|>system\n" + system_msg + "<|im_end|>\n<|im_start|>user\n" + user_msg + "<|im_end|>\n<|im_start|>assistant\n" + assistant_msg + "<|im_end|>"}

print(f"Loading dataset from {DATA_FILE}...")
dataset = load_dataset("json", data_files=DATA_FILE, split="train")
print(f"Dataset size: {len(dataset)} samples")
dataset = dataset.map(format_training_sample, remove_columns=dataset.column_names)
print("Dataset ready!")

In [ ]:
# Cell 9: Setup trainer
config = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    packing=False,  # Disabled for stability
    fp16=True,  # Use fp16 instead of bf16 for T4
    gradient_checkpointing=True,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    warmup_steps=50,
    optim="paged_adamw_8bit",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=config,
    train_dataset=dataset,
    processing_class=tokenizer,
)
print("Trainer ready!")

In [ ]:
# Cell 10: START TRAINING
print("=" * 60)
print("Starting training...")
print(f"Start time: {datetime.now()}")
print("=" * 60)

trainer.train()

print("\n" + "=" * 60)
print("Training complete!")
print(f"End time: {datetime.now()}")
print("=" * 60)

In [ ]:
# Cell 11: Save model
print("Saving model...")
trainer.save_model(f"{OUTPUT_DIR}/final")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/final")
print(f"Model saved to: {OUTPUT_DIR}/final")

# List saved files
!ls -la {OUTPUT_DIR}/final/

In [ ]:
# Cell 12: Download model
# Zip and download
!cd /kaggle/working && zip -r qwen-bharatbuild-finetuned.zip qwen-bharatbuild/final

print("\nModel zipped! Download from the Output section on the right panel.")
print("Or click: Output -> qwen-bharatbuild-finetuned.zip -> Download")